In [2]:
import pandas as pd
import numpy as np

In [3]:
url = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv"
data = pd.read_csv(url)


In [4]:
data = data[[
'Make',
'Model',
'Year',
'Engine HP',
'Engine Cylinders',
'Transmission Type',
'Vehicle Style',
'highway MPG',
'city mpg',
'MSRP']]

In [5]:
data.columns = data.columns.str.replace(' ', '_').str.lower()
data.rename(columns={'msrp':'price'}, inplace=True)
data.fillna(0, inplace=True)

## Question 01

In [6]:
data.apply('mode')

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,Chevrolet,Silverado 1500,2015,200.0,4.0,AUTOMATIC,Sedan,24,17,2000


## Question 02

In [7]:
data.corr()

[10/03/23 14:20:19] WARNING  C:\Users\joao.fonseca\AppData\Local\Temp\ipykernel_20576\2627137660.py warnings.py:109
                             :1: FutureWarning: The default value of numeric_only in DataFrame.corr                
                             is deprecated. In a future version, it will default to False. Select                  
                             only valid columns or specify the value of numeric_only to silence                    
                             this warning.                                                                         
                               data.corr()                                                                         
                                                                                                                   

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


city_mpg and highway_mpg

## Question 03

In [8]:
mean = data.price.mean()
data['above_average'] = (data.price > mean).astype(int)

In [9]:
pip install scikit-learn==0.24.1

Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.model_selection import train_test_split


[10/03/23 14:20:25] WARNING  c:\Users\joao.fonseca\AppData\Local\pypoetry\Cache\virtualenvs\risk-pi warnings.py:109
                             peline-RsNnFdok-py3.9\lib\site-packages\sklearn\utils\multiclass.py:14                
                             : DeprecationWarning: Please use `spmatrix` from the `scipy.sparse`                   
                             namespace, the `scipy.sparse.base` namespace is deprecated.                           
                               from scipy.sparse.base import spmatrix                                              
                                                                                                                   

In [11]:
#X = data.drop(['above_average'], axis=1)
#y = data.above_average

In [23]:
df_train_full , df_test = train_test_split(
    data,test_size=0.2, random_state=42)
df_train, df_val = train_test_split(
    df_train_full, test_size=0.25, random_state=42)

In [24]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
del df_train['above_average']
del df_val['above_average']

In [25]:
from sklearn.metrics import mutual_info_score

In [28]:
categorical = ['make', 'model','transmission_type', 'vehicle_style']
numerical = ['year', 'engine_hp', 'engine_cylinders','transmission_type', 'price','above_average']

In [29]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.above_average)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head(10))


,MI
model,0.460994
make,0.238724
vehicle_style,0.083390
transmission_type,0.020884


## Question 04

In [30]:
from sklearn.feature_extraction import DictVectorizer

In [31]:
dv = DictVectorizer(sparse=False)
train_dict = df_train_full.to_dict(orient='records')
dv.fit(train_dict)

train_dict= df_train.to_dict(orient='records')
X_train = dv.transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [32]:
X_val

array([[0.000e+00, 2.300e+01, 4.000e+00, ..., 0.000e+00, 0.000e+00,
        2.015e+03],
       [0.000e+00, 1.700e+01, 6.000e+00, ..., 0.000e+00, 0.000e+00,
        2.015e+03],
       [0.000e+00, 2.200e+01, 4.000e+00, ..., 1.000e+00, 0.000e+00,
        2.005e+03],
       ...,
       [0.000e+00, 1.200e+01, 6.000e+00, ..., 0.000e+00, 0.000e+00,
        2.003e+03],
       [0.000e+00, 1.400e+01, 8.000e+00, ..., 0.000e+00, 0.000e+00,
        2.015e+03],
       [0.000e+00, 1.800e+01, 6.000e+00, ..., 1.000e+00, 0.000e+00,
        2.015e+03]])

In [33]:
from sklearn.linear_model import LogisticRegression

[10/03/23 14:21:35] WARNING  c:\Users\joao.fonseca\AppData\Local\pypoetry\Cache\virtualenvs\risk-pi warnings.py:109
                             peline-RsNnFdok-py3.9\lib\site-packages\sklearn\utils\optimize.py:18:                 
                             DeprecationWarning: Please use `line_search_wolfe2` from the                          
                             `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace                 
                             is deprecated.                                                                        
                               from scipy.optimize.linesearch import line_search_wolfe2,                           
                             line_search_wolfe1                                                                    
                                                                                                                   

                    WARNING  c:\Users\joao.fonseca\AppData\Local\pypoetry\Cache\virtualenvs\risk-pi warnings.py:109
                             peline-RsNnFdok-py3.9\lib\site-packages\sklearn\utils\optimize.py:18:                 
                             DeprecationWarning: Please use `line_search_wolfe1` from the                          
                             `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace                 
                             is deprecated.                                                                        
                               from scipy.optimize.linesearch import line_search_wolfe2,                           
                             line_search_wolfe1                                                                    
                                                                                                                   

In [47]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [42]:
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [43]:
y_pred_small = model.predict_proba(X_val)[:, 1]

In [44]:
y_pred_small

array([6.87097466e-48, 1.00000000e+00, 5.01382435e-81, ...,
       1.97278901e-73, 1.00000000e+00, 1.00000000e+00])

In [48]:
decision = (y_pred_small >= 0.5)
decision.mean()

0.27780109106168693

In [52]:
y_pred = (y_pred_small >= 0.5)

In [53]:
y_pred

array([False,  True, False, ..., False,  True,  True])

In [51]:
y_val

array([0, 1, 0, ..., 0, 1, 1])

In [57]:
y_val.shape

(2383,)

In [61]:
(y_val == y_pred).mean()

0.9995803608896349

In [59]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [60]:
accuracy

0.9995803608896349

## Question 05